In [ ]:
cd ~/Ali/FBB/Propensity

In [ ]:
import os
n_cpu= os.cpu_count() print(n_cpu)

In [ ]:
import pymysql 
import pandas as pd 
import numpy as np
import seaborn as sns 
import feature_engine 
import warnings
import mysql.connector
import inspect
import matplotlib.pyplot as plt warnings.filterwarnings ('ignore')
pd.set_option('display.max_columns', 40) 
pd.set_option('display.width', 120) 
from sqlalchemy import create engine 
from datetime import datetime 
import timeit
start_time = timeit.default_timer()

In [ ]:
ai = {}
ai['user'] = 'user'
ai['password'] = 'pwd'
ai['name'] = 'AI'
ai['host'] = '10.10.10.10'
ai['port'] = 3333

prod = {}
prod['user'] = 'user'
prod['password'] = 'pwd'
prod['name'] = 'AI'
prod['host'] = '10.10.10.10'
prod['port'] = 3333

In [ ]:
def run_query(cred, db,query):
  try:
    mydb = mysql.connector.connect(
          host=cred['host'],
          port=cred['port'],
          user=cred['user'],
          password=cred['password'],
          database=db
    )
    df = pd.read_sql(query,mydb) 
    mydb.close()
    return df#close the connection 
  except Exception as e:
    print(str(e))

def update_query(cred, db,query):
  try:
    mydb = mysql.connector.connect(
          host=cred['host'],
          port=cred['port'],
          user=cred['user'],
          password=cred['password'],
          database=db
    )
    mycursor mydb.cursor()
    mycursor.execute(query)
    mydb.commit()
    mydb.close()
    return print(mycursor.rowcount, "record(s) affected") 
  except Exception as e:
    print(str(e))

In [ ]:
script_pars ={
      'explanation': 'Model seems fine on production, on lookup prediction there are some flaws, eliminated all pivot and btn variables. Seems like NA pivots also effect', 
      'discard_btnh': 1,
      'target': 'is_sale',
      'discard_pivot': 1,
      'sme_limit':' limit 10000000', #Use it if you want to limit nrows for sme query. Otherwise just ''
      'database': 'smexplorerdata_fbb', #sme database
      'table_to_use': 'smexplorerdata_fbb.fbb_ob_sme_view', #name of the sme
      'neg_pos_ratio': 2, #This is for balancing the dataset. 2 means proportion of target to 1 will be 2:1, and 1 ratio will be 33% 
      'val_split': 'lastN', #Determines the type of validation data split. Shuffle will use a stratified train_test_split
                            # 2nd option is 'lastN'. It will spare the last N days as validation data
      'shuffle_test_size': 0.1, #If you are using shuffle, determine the validation data share. Use to skip it and use all data
      'lastn_days': 20, #Last N days parameter for split option lastN
      'min_day_pct': 0.7, #When you are using last days, you can set a parameter that accidentally sets a large portion of data as validation data
                          #In order to prevent that, you can set this parameter, it looks at distinct days in data, and sets a minimum day limit for validation data beginning 
      'dropna_thresh': 0.5, #Set a threshold for dropping columns having greater null percentage than that value
      'dropcor_thresh': 0.7, #Set a threshold to drop correlated numerical features having correlation coefficient larger than that
      'feature_selection': 1, #If you want to apply a RFECV feature selection step to shrink column size before modeling
      #'rfecv_est': dt, # Which estimator do you want to use for RFECV. dt for decision tree, lgb for lightgbm, pi is inactive due to library issues now but permutation imp 
      'rfecv_min_feat': 50, #How many features will remain in dataset afterwards as a minimum
      'run_type': 'prod', #adds a suffix to models folder, you can use 'prod' or 'test'
      'preferred_folder_name': '', #If you have a folder name containing pickle files you want to make predictions write it.'' is suggested, to get latest created one on run type 
      'top_n_models': 3 #Among all voter models, it chooses top_n models to get average probability
}

In [ ]:
#we have a large amount of variables, so we need to handle them manually in preprocessing.
#this csv file keeps variable names, their real dtypes and decide whether they should be discarded or not 
var_df = pd.read_csv('varlist_fbb_new.csv', na_values = '', delimiter = ', ')
var_df['name'] = var_df['name'].str.lower()
var_df.loc[var_df ['name'].isin(['calldate', 'conn_id']), 'discard'] = 0

if script_pars['discard_btnh'] == 1:
  var_df.loc[var_df['name'].isin([col for col in var_df['name'].unique() if col.startswith('btnh_')]), 'discard'] = 1
if script_pars['discard_pivot'] == 1:
  var_df.loc[var_df['name'].isin([col for col in var_df['name'].unique() if col.startswith('pivot_')]), 'discard'] = 1

var_df.loc[var_df['name'].isin(['feata', 'featb', 'featc', 'featd', 'feate']), 'discard'] = 1

var_drop = list(var_df ['name'][var_df['discard'] ==1])

var_df.drop(var_df.loc[var_df['name'].isin(var_drop) ==True].index, inplace = True)
var_df.reset_index(drop=True, inplace=True)
print(var_df.head())

str_vars=", ".join(var_df['name']) 
print(str_vars)

In [ ]:
#your query
query = """select """ + str_vars + """ from """ + script_pars['table_to_use'] + """ where calltime >= '2022-12-01' and is_sale is not null and (calltime not between '2023-02-05' and '2023-02-20)
and !isnull(agent_id) and agent_id <>'' and agent_id <> ' ' and isrelevant = 1 and proje_tipi in ('Retention', 'Retention_") """ +script_pars['sme_limit']+""" ;""" 

pre_df = run_query(ai, script_pars['database'], query)
print(pre_df.head (5))

In [ ]:
pre_df = pre_df.replace('NA', np.nan)
pre_df = pre_df.replace("NULL", np.nan) 
pre_df.columns= pre_df.columns.str.lower()
var_drop = [col.lower() for col in var_drop]

pre_df.drop(columns = [col for col in var_drop if col in pre_df.columns], axis=1, inplace=True)
#var_df['type'] = np.where(war_df['name'].isin([xxxx]), 'category', var_df['type'])
var_df['type'] = np.where(var_df ['type'].isin(['int']), 'Int64', var_df['type'])
print(pre_df.isna().sum())
print(pre_df.head())

In [ ]:
#cardinality check, it can also be automated
print(pre_df.nunique().sort_values (ascending=False).head (40))
print(len(pre_df))
#print(variable5.value_counts().sort_values (ascendint=False))

In [ ]:
pre_df_cols = pre_df.columns
varlist = list(var_df['name'].str.lower())
print(len(pre_df_cols))

var_df.loc[var_df['name'].isin(['cmpg_acik_sikayet_kaydi', 'cmpg_risk_skoru']), 'type'] = 'category'

#changing python dtypes and if they are different than our list, change dtype. Float thing was not necessary, but after so many errors inside pipe, i needed to do it 
for i in range(len(pre_df_cols)):
  print(i)
  if var_df.loc[i, 'type'] == 'datetime64":
    #main_df[main_df_cols[i]] = pd.to_datetime (main_df[main_df_cols[i]], format = var_df.loc[i, 'timetype'], errors = 'coerce')
    pass
  if var_df.loc[i, 'type'] == 'Int64":
    print (pre_df_cols[i])
    print(var_df.loc[i, 'type'])
    pre_df [pre_df_cols[i]] = pre_df[pre_df_cols[i]].astype('float')
    pre_df[pre_df_cols[i]] = pre_df[pre_df_cols[i]].astype(var_df.loc[i, 'type'])
  else:
    print (pre_df_cols[i])
    print(var_df.loc[i, 'type'])
    pre_df [pre_df_cols[i]] = pre_df[pre_df_cols[i]].astype (var_df.loc[i, 'type'])

In [ ]:
print(round(pre_df.isnull(). mean()*100,2).sort_values (ascending = False))

In [ ]:
if [i for i in ['feata', 'featb'] if i in pre_df.columns]:
  print("It is in columns")
  
  ### !!!!!!! This is an important concept with 2 outcomes to be used in future. This cell is only for test purpose. 
  ### Replace is not workint with INPLACE parameter anymore, you need to specify it explicitly
  ### And if the DTYPE is not OBJECT, replacing NP.NAN value with a fixed string value is not working. You need to change DTYPE 
  pre_df_test = pre_df.copy()
  pre_df_test[['feata', 'featb']] = pre_df_test[['feata', 'featb']].replace(np.nan, 'NA')
  print (pre_df_test[['feata', 'featb']].head(10))

  pre_df_test[['feata', 'featb']] = pre_df_test[['feata', 'featb']].astype('object') 
  pre_df_test[['feata','featb']] = pre_df_test[['feata', 'featb']].replace(np.nan, 'NA')
  print(pre_df_test[['feata', 'featb']].head(10))

  ### Here is the 2nd variatoin, using fillna. But it needs to see it as a new category
  ### And also add_category is only using series, one column each time. So, i used a lambda funtion to apply it properly
  #These features are filled with NA because features are populated only if there is a cancellation application. So, missing values have meaning here
  pre_df[['feata', 'featb']] = pre_df[['feata','featb']].apply(lambda x: x.cat.add_categories('NA').fillna('NA')) 
  print(pre_df[['feata', 'featb']].head())

In [ ]:
print(round(pre_df.isnull().mean()*100,2).sort_values(ascending = False))

In [ ]:
### NA value filling is a complicated issue. So far i have only filled the columns which NaN values have meaning, and not missing information.
### For other ones, according to the requirements of algorithms, i will fill them with median or most frequent values inside trainin pipelines (Leakage issues) 
### And as a first step, during analysis of data, such columns should be determined, the ones that NaN values has a meaning, can be replaced as a category.

In [ ]:
### Since it is random over/under sampling, it is not deeply connected to preprocessing steps we will apply. It can be before or after preprocessing.
### I was just thinking to calculate null value percentages etc with full population, but it may be better to solve imbalance first.
### Because we are deailing with null values etc to make model better. But, when we do them first, then solve imbalance. randomly, we may come across with new Na issue etc. 
import math

#imbalance check
pre_df_copy = pre_df.copy()
pos_pre_df = pre_df [pre_df['is_sale']==1].reset_index(drop=True)
neg_pre_df = pre_df[pre_df['is_sale']==0].reset_index(drop=True) 
num_positive = pos_pre_df.shape[0]
num_negative neg_pre_df.shape[0]
neg_pos_ratio= script_pars['neg_pos_ratio']
print(len(pos_pre_df))
print(len(neg_pre_df))
main_df = pre_df.copy()

np.random.seed(35) #1000 first
#sampling_idx = np.random.randint(0, neg_pre_df.shape[0], math.floor(num_positive*neg_pos_ratio)).tolist()
sampling_idx= np.random.choice(range(neg_pre_df.shape[0]), math.floor(num_positive*neg_pos_ratio), replace = False).tolist()

sampling_idx.sort()
sub_neg_df = neg_pre_df.iloc[sampling_idx, :]

outsamp_neg_df = neg_pre_df.iloc[~neg_pre_df.index.isin(sampling_idx), :] 
#outsamp_neg_df = neg_pre_df.iloc[outsamp_ind, :]

print(len(sub_neg_df))
#outsamp_neg_df = neg_pre_df[~neg_pre_df.index.isin(sampling_idx)] 
print(len(outsamp_neg_df))

main_df = pos_pre_df.append(sub_neg_df).reset_index(drop=True)

#shuffling dataset 5 times to randomise output var again
for i in range(5):
  idx = np.random.permutation(main_df.index)
  main_df = main_df.reindex(idx)

main_df = main_df.reset_index(drop=True)
outsamp_neg_df = outsamp_neg_df.reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

if (script_pars['val_split'] == 'shuffle') and (script_pars['shuffle_test_size'] > 0):
  
  main_df, val_df = train_test_split(main_df, test_size = script_pars['shuffle_test_size'], stratify = main_df['is_sale'], random_state = 35) 
  main_df['is_val'] = 0
  val_df['is_val'] = 1
  outsamp_neg_df ['is_val'] = 2
  main_df = main_df.append(val_df, ignore_index = True)
  main_df = main_df.append(outsamp_neg_df, ignore_index = True)

elif script_pars['val_split'] == 'lastN':
  max_dt = max(main_df['calldate'])
  min_dt = min(main_df['calldate'])
  thr_day = max_dt - timedelta(days = script_pars['lastn_days'])

  #Active days list
  window_days = main_df['calldate'].nunique()
  
  # Thr_day looks at only active call_dates, and sets a minimum threshold to them according to x% of the active days. 
  thr_day_limit = sorted(main_df['calldate'].unique())[math.ceil (window_days*script_pars['min_day_pct'])+1] 
  print(thr_day)
  print(thr_day_limit)
  print(max(thr_day, thr_day_limit))
  main_df['is_val'] = 0
  main_df['is_val'][main_df['calldate']>=max(thr_day, thr_day_limit)] = 1

  outsamp_neg_df['is_val'] = 2
  main_df = main_df.append(outsamp_neg_df, ignore_index = True)

else:

  main_df['is_val'] = 0
  outsamp_neg_df['is_val'] = 2
  main_df = main_df.append(outsamp_neg_df, ignore_index = True)

print(main_df['is_val'].value_counts()) 

In [ ]:
#setting a NA threshold and dropping vars according to that. Also fix, empty and duplicate columns are deleted. 
main_df.dropna(axis=1, thresh = int(script_pars['dropna_thresh']*(main_df.shape[0])), inplace=True)

df_y=main_df[['is_sale', 'is_val']]
print(df_y['is_val'].value_counts())
main_df.drop(columns = ['is_sale'], axis= 1, inplace=True)

fix_cols = []
for col in [col for col in main_df.columns if col not in ['is_val']]:
  if main_df[col].nunique() == 1:
    fix_cols.append(col)
  else:
    pass

emp_cols = []
for col in [col for col in main_df.columns if col not in ['is_val']]: 
  if main_df[col].count() > 0:
    pass
  else:
    emp_cols.append(col)

print(len(fix_cols))
print(len(emp_cols))
print(main_df.info())

def getDuplicateColumns(df): 
  duplicateColumnNames = set() 
  duplicateColumnDict = dict() 
  for x in range(df.shape[1]):
    col = df.iloc[:,x]
    for y in range (x+1, df.shape[1]):
      otherCol = df.iloc[:,y]
      if col.equals(otherCol):
        duplicateColumnNames.add(df.columns.values[y])
        duplicateColumnDict[df.columns[x]] = df.columns[y]
  return list(duplicateColumnNames), duplicateColumnDict

dupColNames, dupColDict = getDuplicateColumns(main_df)

for key, value in dupColDict.items():
  print("key={}, value = {}".format(key, value)) 
dropcols = list()

dropcols.extend([x for x in fix_cols if x not in dropcols]) 
dropcols.extend([x for x in emp_cols if x not in dropcols]) 
dropcols.extend([x for x in dupColNames if x not in dropcols]) 
print(dropcols)

main_df.drop(columns= dropcols, axis=1, inplace=True)

var_df = var_df[var_df['name'].isin(list(main_df.columns) + list(df_y.columns)) == True]

In [ ]:
### Decided to fill all categorical NaN values with NA category. 
df_y_final = df_y[['is_sale', 'is_val']]
print(df_y_final['is_val'].value_counts())

main_df.drop(columns = ['featx'], inplace=True)

print(main_df.dtypes.astype(str).value_counts())

date_cols = list(main_df.select_dtypes(include = ['datetime64']).columns)

num_cols = list(main_df.select_dtypes(include = ['float', 'int', 'int64', 'Float64', 'Int64', 'float64']).columns)
cat_cols = list(main_df.select_dtypes(exclude = ['float', 'int', 'int64', 'Float64', 'Int64', 'float64', 'datetime64']).columns) 

num_cols.remove('is_val')

main_df.drop(columns = date_cols, axis=1, inplace=True)

print(len(num_cols)) 
print(len(date_cols))
print(len(cat_cols))

main_df1 = main_df.iloc[:500000,]
main_df2= main_df.iloc[500000:,]
print(main_df2.info())
#!! category dtype do not let you replace or fill np.nan's with new category value, that is why i switched to str type 
#divided df into 2 as it was having memory error
for col in cat_cols:
  main_dfl[col] = main_dfl[col].astype('object')
  main_dfl[col] = main_df1[col].replace(np.nan, 'NA')

for col in cat_cols:
  main_df2[col] = main_df2[col].astype('object') 
  main_df2[col] = main_df2[col].replace(np.nan, 'NA')

main_df = main_dfl.append(main_df2, ignore_index = True)
print(main_df.dtypes.astype(str).value_counts())

In [ ]:
print(main_df['is_val'].value_counts())
val_x = main_df [main_df['is_val']==1]
outsamp_df = main_df[main_df['is_val']==2] 
main_df = main_df[main_df['is_val']==0]

val_y = df_y_final[df_y_final['is_val']==1] 
outsamp_y= df_y_final[df_y_final['is_val']==2] 
df_y_final = df_y_final[df_y_final['is_val']==0]

print(df_y_final['is_val'].value_counts())

val_x.reset_index(drop=True, inplace=True) 
main_df.reset_index(drop=True, inplace=True)
outsamp_df.reset_index(drop=True, inplace=True)

val_y.reset_index(drop=True, inplace=True) 
df_y_final.reset_index(drop=True, inplace=True) 
outsamp_y.reset_index(drop=True, inplace=True)

print(main_df['is_val'].value_counts()) 
print(df_y_final['is_val'].value_counts()) 
print(outsamp_y['is_val'].value_counts())

main_df.drop(columns = ['is_val'], inplace=True) 
val_x.drop(columns = ['is_val'], inplace=True) 
outsamp_df.drop(columns = ['is_val'], inplace=True)

df_y_final.drop(columns = ['is_val'], inplace=True) 
val_y.drop(columns = ['is_val'], inplace=True) 
outsamp_y.drop(columns = ['is_val'], inplace=True)

In [ ]:
# This part is used to eliminate numerical columns that lead to multicolliunearity. If a pair is found to have higher corr value than threshold, the one having more cumulative 
# correlation with others is eliminated (Only for input data)
from statistics import mean

def get_redundant_pairs(df):
  pairs_to_drop = set()
  cols = df.columns
  for i in range(0,df.shape[1]):
    for j in range(0,i+1):
      pairs_to_drop.add((cols[i], cols[j]))
  return pairs_to_drop

def get_triangle(df):
  au_corr = df.corr().abs().unstack()
  labels_to_drop = get_redundant_pairs(df)
  au_corr = au_corr.drop(labels=labels_to_drop)
  #print (au_corr)
  au_corr = au_corr.sort_values (ascending=False)
  return au_corr

left_cols = num_cols.copy()
corr_cols = []
elim_rows = []
mean_dict = {}

row = 0
while get_triangle(main_df[left_cols])[0:1].values > script_pars['dropcor_thresh']:

  #print (len(left_cols))
  sub_df = get_triangle(main_df[left_cols])
  filter_df_1 = [t for t in sub_df.index if (t[0] == sub_df.index[0][0]) or (t[1] == sub_df.index[0][0])] 
  filter_df_2 = [t for t in sub_df.index if (t[0] == sub_df.index[0][1]) or (t[1] == sub_df.index[0][1])] 
  #print(np.nanmean(sub_df[filter_df_1].values))
  if np.nanmean(sub_df[filter_df_1].values) > np.nanmean(sub_df[filter_df_2].values):
    corr_cols.append(sub_df[0:1].index[0][0]) 
    left_cols.remove(sub_df[0:1].index[0][0])
  else:
    corr_cols.append(sub_df[0:1].index[0][1])
    left_cols.remove(sub_df[0:1].index[0][1])
  elim_rows.append(sub_df[0:1])
  mean_dict[row]=[(sub_df[0:1].index[0][0], np.nanmean(sub_df[filter_df_1].values)), (sub_df[0:1].index[0][1], np.nanmean(sub_df[filter_df_2].values))] 
  row = row + 1
  #print(len(left_cols))

print(corr_cols)
print(elim_rows)
print(mean_dict)

In [ ]:
num_cols = [x for x in num_cols if x not in corr_cols]
#print(sub_df[sub_df.apply(lambda x: 'featxx' in x)])
print(num_cols)

In [ ]:
main_df.drop(corr_cols, axis=1, inplace=True) 
outsamp_df.drop(corr_cols, axis=1, inplace=True)
val_x.drop(corr_cols, axis=1, inplace=True)

In [ ]:
# These 5 fields below is for calculating cramers_v scores between categorical variables. So that we can define correlated categorical ones, and eliminate if needed. 
column_headers = [cat_cols[x].split()[-1] for x in range(len(cat_cols))] 
print(column_headers)

main_df_cramer = main_df.copy()

def gnumeric_func(data, columns):
  data[columns] = data[columns].apply(lambda x: pd.factorize(x)[0])
  # return data

gnumeric_func(main_df_cramer, column_headers)

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import pandas as pd
final_df= pd.DataFrame(columns=('column1', 'column2', 'cramers_V')) 
#Due to memory issues, set these to 100k

def cramers_v(x,y, main_df_cramer):

  """
  x and y are 2 columns to be compared related to cramer's score
  main_df_cramer is calculated with above gnumeric_func, and it is used here
  """
  confusion_matrix = pd.crosstab(main_df_cramer[x].iloc[:100000],main_df_cramer[y].iloc[:100000])
  chi2 = chi2_contingency(confusion_matrix)[0]
  n = confusion_matrix.sum().sum()
  phi2 =chi2/n
  r, k = confusion_matrix.shape
  phi2corr = max(0,phi2-((k-1)*(r-1))/(n-1))
  rcorrr = r -((r-1)**2)/(n-1)
  kcorr = k-((k-1)**2)/(n-1)
  return np.sqrt(phi2corr/min((kcorr-1), (rcorr-1)))
i = 0

used_cols = []

for col1 in column_headers:
  if (col1 !='on_off') & (col1 not in used cols):
    #print(col1)
    used_cols.append(col1) 
    #print (used_cols)
    
    for col2 in column_headers:
      if (col2 !='on_off' and col1!=col2) & (col2 not in used_cols):
        x = cramers_v(col1, col2, main_df_cramer)
        final_df.loc[i]=[col1, col2,x]
        i=i+1


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline

# plt.imshow(final_df, cmap='hot', interpolation='nearest')
# plt.show()

#final_df.plot(x='c1')
temp_df= final_df.pivot(index='columnl', columns='column2', values='cramers_V') 
pyplot.figure(figsize=(20,20))
hm=sns.heatmap(temp_df, vmin=0, vmax=1, linewidths=0.1)

hm.figure.savefig("output.png")


In [ ]:
print(main_df_cramer[used_cols].nunique())
uniq_df = pd.DataFrame(main_df_cramer[used_cols].nunique(), columns = ['uniq_cnt']) 
uniq_df.reset_index(inplace=True)
uniq_df = uniq_df.rename(columns = {'index':'name'})
print(uniq_df)

In [ ]:
final_merged = pd.merge(final_df, uniq_df,left_on = 'column1', right_on='name', how='left') 
final_merged = final_merged.drop(columns = ['name'])
final_merged = final_merged.rename(columns = {'uniq_cnt': 'uniq_cnt_left'})

final_merged2 = pd.merge(final_merged, uniq_df, left_on = 'column2', right_on = 'name', how='left') 
final_merged2 = final_merged2.drop(columns = ['name'])
#print(final_merged2)
final_merged2.sort_values(by=['cramers_V', 'column1'], ascending=False).iloc[:50]

In [ ]:
main_conn_df = main_df.loc[:,'conn_id']
outsamp_conn_df = outsamp_df.loc[:, 'conn_id']
val_conn_df = val_x.loc[:,'conn_id']

main_df.drop(columns = ['conn_id'], axis=1, inplace=True) 
outsamp_df.drop(columns=['conn_id'], axis=1, inplace=True) 
val_x.drop(columns = ['conn_id'], axis=1, inplace=True)

cat_cols.remove('conn_id')

In [ ]:
if script_pars['feature_selection'] == 1:
  from sklearn.tree import DecisionTreeClassifier
  from sklearn.inspection import permutation_importance
  # IF YOU WILL USE PERMUTATION IMP INSIDE RFECV YOU NEED TO GET ELIS VERSION. IT IS SUITABLE, NOT SKLEARN VERSION 
  #from eli5.sklearn import PermutationImportance
  from sklearn.linear_model import LogisticRegression
  from sklearn.pipeline import Pipeline
  from sklearn.compose import ColumnTransformer
  from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
  from sklearn.feature_selection import RFECV
  from lightgbm import LGBMClassifier
  from sklearn.feature_selection import mutual_info_classif
  from sklearn.feature_selection import Generic UnivariateSelect

  dt = DecisionTreeClassifier(random_state = 0)
  lr = LogisticRegression()
  lgb = LGBMClassifier(random_state = 23, max_depth = 6, n_estimators = 300)

  main_df_copy = main_df.copy()

  #check it, it may not work as expected
  main_df_copy[cat_cols] = main_df_copy[cat_cols].replace(np.nan, 'NULL')


  orden = OrdinalEncoder()
  main_df_copy[cat_cols]= orden.fit_transform(main_df_copy[cat_cols])
  main_df_copy[num_cols] = main_df_copy[num_cols].fillna(main_df_copy[num_cols].median())

  ###It will be used if permutation feature importance is needed to applied into pipeline 
  #pi = PermutationImportance (dt, cv=4, random_state = 0)

  #rfecv has a problem with pipeline recognition
  #cat_transformer = Pipeline(steps = [('enc', OrdinalEncoder())])
  #num_transformer = Pipeline(steps = [('scl', MinMaxScaler (feature_range = (0,1)))])
  #preprocessor = ColumnTransformer (transformers = [('cat', 'passthrough', cat_cols), ('num', 'passthrough', num_cols)], remainder = 'passthrough')

  #all_pipe = Pipeline(steps = [('prep', preprocessor), ('est', lgb)])

  #rfe = RFECV(estimator = pi, cv = 4, min_features_to_select = 80, scoring = 'roc_auc', step = 1, n_jobs =12)
  rfe = RFECV(estimator = dt,cv = 4, min_features_to_select = script_pars['rfecv_min_feat'], scoring = 'roc_auc', step = 1, n_jobs =16) 
  rfe.fit(main_df_copy, df_y_final)

  """
  mutual_inf = mutual_info_classif (main_df, df_y_final, random_state = 10)
  generic = GenericUnivariateSelect(score_func = mutual_info_classif, mode = 'k_best', param=100) 
  main_df_trans = generic.fit_transform(main_df, df_y_final)
  print(main_df['var8'][main_df['var8'].isnull()])
  """

  ### This part is to restructure num and cat cols after elimination of first part features

  main_df_copy[cat_cols] = orden.inverse_transform(main_df_copy[cat_cols])
  cols_left = main_df_copy.columns[rfe.get_support()].values

  main_df = pd.DataFrame(main_df, columns= cols_left)
  outsamp_df = pd.DataFrame(outsamp_df, columns = cols_left) 
  val_x = pd.DataFrame(val_x, columns = cols_left)

  num_vars= num_cols.copy()
  num_cols = []
  cat_vars= cat_cols.copy() 
  cat_cols = []

  for col in cols_left:
    if col in cat_vars:
      main_df [col] = main_df [col].astype('object') 
      cat_cols.append(col)
    else:
      pass

  for col in cols_left:
    if col in num_vars:
      main_df [col] = main_df [col].astype('float') 
      num_cols.append(col)
    else:
      pass
  print(num_cols) 
  print(cat_cols)

if script_pars['feature_selection'] == 0:
  cat_cols = ['feata', 'featb', 'featc']
  num_cols = ['featd', 'feate featf'] 
  
  cols_left = cat_cols + num_cols

  main_df = pd.DataFrame(main_df, columns= cols_left)
  outsamp_df = pd.DataFrame(outsamp_df, columns = cols_left)
  val_x = pd.DataFrame(val_x, columns = cols_left)

  num_vars = num_cols.copy()
  num_cols = []
  cat_vars= cat_cols.copy()
  cat_cols = []

  for col in cols_left:
    if col in cat_vars:
      main_df [col] = main_df[col].astype('object') 
      cat_cols.append(col)
    else:
      pass
  for col in cols_left:
    if col in num_vars:
      main_df [col] = main_df[col].astype('float') 
      num_cols.append(col)
    else:
      pass

In [ ]:
def col_ind_get(prep, all cols, num_cols, cat_cols, enc_cols):
  """
  This function looks at the structure of Pipeline transformation, and determines which features and feature indices are related to which pipeline sub process
  It looks at the preprocessor instance, which is a column transformer consisting of cat and numerical transformers
  I am still not sure about enc_cols parameter, i forgot why i added it. But it is sth like encoded variables, will not be encoded etc.
  Below you are seeing transformers i 2, which retrieves which column is used for which transformer.
  Aim of this function is to keep order of columns after transformation, preprocessing. It will keep ordered names and indices, and it will be helpful for reverse transformation after onehot etc 
  """
  pre_all_cols = []
  pre_cat_cols = []
  pre_cat_ind = []

  cat_colss = list(set(cat_cols) - set(enc_cols))

  for i in range(len(prep.transformers)):
    pre_all_cols.extend(prep.transformers[i][2])

  pre_cat_cols = [item for item in pre_all_cols if item in cat_colss]
  pre_cat_ind = [i for i in range(len(pre_all_cols)) if pre_all_cols[i] in cat_colss] 
  pre_num_cols = [item for item in pre_all_cols if item in num_cols]
  pre_num_ind = [i for i in range(len(pre_all_cols)) if pre_all_cols[i] in num_cols]

  return pre_all_cols, pre_cat_cols, pre_cat_ind, pre_num_cols, pre_num_ind

In [ ]:
run_date = datetime.now().strftime("%y%m%d%H%M")

best_results = dict() 
best_params = dict() 
best_models= dict()
best_summary = dict()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from feature_engine.imputation import Categorical Imputer, MeanMedian Imputer
from feature_engine.encoding import RareLabelEncoder
from lightgbm import LGBMClassifier
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, Logistic Regression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from category_encoders.leave_one_out import LeaveOneOutEncoder

ordered_cols = cat_cols + num_cols

main_df2 = main_df.reindex(columns = ordered_cols)
outsamp_df = outsamp_df.reindex(columns = ordered_cols)
val_x = val_x.reindex (columns=ordered_cols)

df_y_final[script_pars['target']] = df_y_final[script_pars['target']].astype('int') 
outsamp_y[script_pars['target']] = outsamp_y[script_pars['target']].astype('int') 
val_y[script_pars['target']] = val_y[script_pars['target']].astype('int')
####!!!!! Why i was using enc_cols!!!!!!11
enc_cols = []


models = dict() 

paramslogi = {}

paramsrfc = {
  'est_rfc_max_depth': [11,21,31],
  'est_rfc_n_estimators': [25,75,150],
  'est_rfc_min_samples_split': np.linspace(0.1, 0.9, 3, endpoint = True),
  'est_rfc_verbose': [1],
  'est_rfc_random_state': [35],
  'est_rfc_n_jobs': [16]
  #'est_min_samples_leaf': np. linspace (0.1, 0.5, 3, endpoint = True)
  }

paramsgnb = {
  }

paramsxgbc = {
  'est_xgbc_n_estimator': [25,75,150],
  'est_xgbc_max_depth': [11,21,31],
  'est_xgbc_learning_rate': [0.0001, 0.001],
  'est_xgbc_random_state': [35],
  'est_xgbc_verbosity': [1],
  'est_xgbc_n_jobs': [8]
  }

paramsknnc = {
  'est_knn_n_neighbors': [3,5,8]
  }

paramssgdc = {
  'est_sgdc_penalty': ['l1', 'l2', 'elasticnet'],
  'est_sgdc_max_iter': np.linspace (100,700, 3, endpoint=True, dtype = 'int'),
  'est_sgdc_alpha': [0.0001, 0.001],
  'est_sgdc_loss': ['log_loss'],
  'est_sgdc_random_state': [35],
  'est_sgdc_n_jobs': [16]
  }

paramslgb = {
  'est_lgb_max_depth': [11,21, 31],
  'est_lgb_learning_rate': [0.001,0.0001],
  'est_lgb_n_estimators': [25, 75, 150],
  'est_lgb_num_leaves': np.linspace (11,91,3, endpoint=True, dtype = int),
  'est_lgb_verbosity': [1],
  'est_lgb_random_state': [35],
  'est_lgb_n_jobs': [16]
  }

paramssvc = {
  'est_svc_C': [1,10,100],
  'est_svc_kernel': ['linear', 'rbf'],
  'est_svc_tol': [0.0001, 0.001]
  }

models['lgb'] = [LGBMClassifier(), paramslgb]
models['rfc'] = [RandomForestClassifier(), paramsrfc]
##models['knn'] = [KNeighborsClassifier (algorithm = 'kd_tree', n_jobs=16), paramsknnc]
models['gnb'] = [GaussianNB(), paramsgnb]
models['sgdc'] = [SGDClassifier(), paramssgdc]
models['logi'] = [LogisticRegression (random_state= 35, n_jobs = 16), paramslogi]
models['xgbc'] = [XGBClassifier (tree_method = 'hist', objective = 'binary: logistic'), paramsxgbc] 
##models['svc'] = [SVC (verbose= 1, random_state= 35, probability = True), paramssvc] very slow

for key, value in models.items():
  start_time = timeit.default_timer()
  scorer = ['roc_auc', 'accuracy', 'f1', 'jaccard']
  ### !!!! IDEA FOR MISSING VALUE HANDLING: If a row has missing values for columns like 10% of all, as a threshold. Drop it. Such an idea can be applied before the undersampling process 
  numeric_transformer = Pipeline(steps = [('Mimp', MeanMedianImputer(imputation_method = 'median', variables = num_cols)), 
                                          ('scl', MinMaxScaler (feature_range = (0,1)))])
  # since gaussiannb doesnt want a sparse matrix
  if key 'gnb':
    categoric_transformer = Pipeline(steps = [('Cimp', CategoricalImputer(imputation_method = 'missing', fill_value = 'NA', variables = cat_cols)), 
                                              ('rare', RareLabelEncoder(tol = 0.01, n_categories = 3, replace_with = 'rare', variables = cat_cols)), 
                                              #('Loo', LeaveOneOutEncoder(sigma = 0.1, cols = cat_cols))])
                                              ('hot', OneHotEncoder(handle_unknown = 'ignore', sparse = False))])

  else:
    categoric_transformer = Pipeline(steps = [('Cimp', CategoricalImputer(imputation_method = 'missing', fill_value = 'NA', variables = cat_cols)), 
                                              ('rare', RareLabelEncoder(tol = 0.01, n_categories = 3, replace with = 'rare', variables = cat_cols)), 
                                              #('Loo', LeaveOneOutEncoder(sigma = 0.1, cols = cat_cols))]) 
                                              ('hot', OneHotEncoder(handle_unknown = 'ignore', sparse = True))])

  preprocessor = ColumnTransformer(transformers = [('cat', categoric_transformer, cat_cols),
                                                  ('num', numeric_transformer, num_cols)], remainder = 'passthrough')
  all_pipe = Pipeline(steps = [('prep', preprocessor), (str('est '+key), value[0])])

  pre_all_cols, pre_cat_cols, pre_cat_ind, pre_num_cols, pre_num_ind = col_ind_get(preprocessor, main_df2.columns, num_cols, cat_cols, enc_cols) 
  search_space = value[1]

  #print(pre_all_cols) 
  #print(pre_cat_cols)
  #print(pre_cat_ind)
  #print(pre_num_cols)
  #print(pre_num_ind)
  #print(list(main_df2.columns))
  #print(main_df2.columns)
  #print(main_df2.head(2))

  #for col in cat_cols:
  # print(col)
  # print(main_df2.loc[:, col].value_counts())
  #print(main_df2[num_cols].info())
  #print(df_y_final.head(5))

  ###Using pipeline inside GridSearch is very critical!!!!!!!!!!, it only fits preprocessing on train folds in this structure

  ### When you want to run whole data with best parameters found, you can use refit = True
  ### But like here, when there are multiple scorers, you define one scorer here, to define best model and hyperparameters according to this metric 
  ### For a cv of 5, you will have 5 accuracy and 5 roc scores for both train and test. k=1 may have highest mean accuracy, but k=5 may have roc 
  ### So, you are decidin to use refit variable as decider. And fitting whole data according to that. Getting best model out of it as best estimator 
  ### So, you may not need to fit all dataset again, if you have best estimator here!!!!!!!!! !!!
  ### THIS BEST ESTIMATOR will enable you to use PREDICT and PREDICT_PROBA etc.
  grid_search= GridSearchCV(all pipe, search_space, cv= 5, verbose= 1, refit = 'roc_auc', scoring scorer, return_train_score = True, n_jobs=16)

  # this version is used for CatBoost, when we give it cat features automatically it handles them. But now we are using mixed algos, and already preprocessed data 
  #grid_search.fit(main_df2, y=df_y_final, est_cat_features=pre_cat_ind)
  grid_search.fit(main_df2, y=df_y_final)

  ind = grid_search.best_index_
  print("model = {}".format(key))
  print("train_roc = {}, test_roc = {}".format(grid_search.cv_results_['mean_train_roc_auc'][ind], grid_search.cv_results_['mean_test_roc_auc'][ind])) 
  print("train_accuracy {}, test_accuracy = {}".format(grid_search.cv_results_['mean_train_accuracy'][ind], grid_search.cv_results_['mean_test_accuracy'][ind])) 
  print("train_f1 = {}, test_f1 = {}".format(grid_search.cv_results_['mean_train_f1'][ind], grid_search.cv_results ['mean_test_f1'][ind]))
  print("train_jaccard = {}, test_jaccard = {}".format(grid_search.cv_results_['mean_train_jaccard'][ind], grid_search.cv_results_['mean_test_jaccard'][ind])) 
  print("avg_fit_time = {}".format(grid_search.cv_results_['mean_fit_time'][ind]))
  print("best_params = {}".format(grid_search.best_params_))
  print("best_estimator = {}".format(grid_search.best_estimator_.named_steps[str('est_'+key)]))
  #print("---0.1f minutes---") %((timeit.default_timer()-start_time)/60)
  print("run_start = {}...".format(start_time))
  print("current_time = {}...".format(datetime.now()))

  final_list = []
  ### Print also the foldwise train and test scores, apart from mean values. To see if it has high STD error
  ### std_train_score str_train_roc_auc std_test_roc_auc mean_fit_time std_fit_time mean_score_time std_score_time
  best_params[key] = grid_search.best_params_
  final_list.append(grid_search.best_params_)

  best_results[key] = [grid_search.cv_results_['mean_test_roc_auc'][ind], grid_search.cv_results_['mean_test_accuracy'][ind], grid_search.cv_results_['mean_test_f1'][ind], 
                       grid_search.cv_results_['mean_test_jaccard'][ind]] 
  final_list.append([grid_search.cv_results_['mean_test_roc_auc'][ind], grid_search.cv_results_['mean_test_accuracy'][ind],
                      grid_search.cv_results_['mean_test_f1'][ind], grid_search.cv_results_['mean_test_jaccard'][ind]])
  
  best_models [key] = grid_search.best_estimator_
  final_list.append(grid_search.best_estimator_)

  best_summary[key] = final_list
  with open(os.getcwd()+"/model_results/backup/" + run_date + "-" + key + ".pkl", 'wb') as f:
    pickle.dump(best_summary[key][2], f)

  """
  importances = grid_search.best_estimator_.named_steps['est'].feature_importances_ 
  importances = [float(i) for i in importances]

  print(pre_all_cols)
  print(importances)
  print(list(zip(pre_all_cols, importances)))
  """

In [ ]:
import pickle

os.mkdir(os.getcwd()+"/model_results/" + run_date + "-" + script_pars['run_type'])

for key, value in best_summary.items():
  with open(os.getcwd()+"/model_results/" + run_date + "-" + script_pars['run_type']+ "/" + key + ".pkl", 'wb') as f: 
    pickle.dump(best_summary[key][2], f)

In [ ]:
model_success_df = pd.DataFrame(columns = ['model', 'auc_score', 'accuracy_score', 'fl_score', 'jaccard_score']) 
best_summary_copy = best_summary.copy()

### check all list operations, list = another_list doesnt work, it binds each. Doesnt create an independent list 
for key, value in best_summary.items():
  print(best_summary[key][1])
  temp_list = best_summary[key][1][:] 
  print(temp_list)
  temp_list.insert(0, str(key))
  print(temp_list)
  model_success_df.loc[len(model_success_df)] = temp_list

model_success_df['auc_rank'] = model_success_df['auc_score'].rank(ascending = False, method = 'min')
model success_df['accuracy_rank'] = model_success_df['accuracy_score'].rank(ascending = False, method = 'min') 
model_success_df['f1_rank'] = model_success_df['f1_score'].rank(ascending = False, method = 'min')
model_success_df['jaccard_rank'] = model_success_df['jaccard_score'].rank (ascending = False, method = 'min')
model_success_df['overall_total_rank'] = model_success_df['auc_rank'] + model_success_df['accuracy_rank'] + model_success_df['f1_rank']# + model_success_df['jaccard_rank"] 
model_success_df['overall_rank'] = model_success_df['overall_total_rank'].rank(ascending = True, method = 'min')
model_success_df = model_success_df.sort_values(['overall_rank', 'accuracy_score', 'fl_score'], ascending [True, False, False]).reset_index() 
print(model_success_df)

In [ ]:
top_n_models = script_pars['top_n_models']

voter_models = model_success_df['model'].iloc[:top_n_models].unique()
voter_models.sort()
print(voter_models)

In [ ]:
from glob import glob
preferred_folder_name = script_pars['preferred_folder_name']

if preferred_folder_name == '':

  #model_dirs = [x[0] for x in os.walk (os.getcwd())]
  model_dirs = glob(os.getcwd()+"/model_results/*/", recursive = True)
  model_dirs_filtered = [x for x in model_dirs if script_pars['run_type'] in x]
  model_dirs_filtered.sort(reverse = True)
  model_dir = model_dirs_filtered[0]

  model_list = [s for s in os.listdir(model_dir) if any(x in s for x in voter_models)]

  elif preferred_folder_name != '':
    model_dir = os.getcwd() + "/model_results/" + preferred_folder_name + "/"

  model_list = [s for s in os.listdir(model_dir) if any (x in s for x in voter_models)]

In [ ]:
model_res_dict = {}
model_res_dict['columns'] = model_success_df.columns
for i in model_success_df.index:
  model_res_dict[i] = model_success_df.iloc[i,:]

  with open(model_dir+"/voting_results.txt", 'w') as f:
    print(model_res_dict, file = f)

  with open(model_dir+"/voting_results.pkl", 'wb') as fp:
    pickle.dump(model_res_dict, fp)
    print('dictionary saved successfully to file')
  
  with open(model_dir+"/voting_df.pkl", 'wb') as fp:
    pickle.dump(model_success_df, fp)
    print('dictionary saved successfully to file')

In [ ]:
models = []
for i in range(len(model_list)):
  with open(model_dir+"/"+model_list[i], 'rb') as f:
    model =  pickle.load(f)
    models.append(model)

In [ ]:
print(model_list)
print(model_dirs)
print(model_dirs_filtered) 
print(model_dir)

In [ ]:
predictions = pd.DataFrame()
for i in range(len(models)):
  predictions['proba' + str(i)] = models[i].predict_proba(main_df2)[:,1]
  predictions['predict' + str(i)] = models[i].predict(main_df2)

In [ ]:
predictions['avg_proba'] = predictions[[col for col in predictions.columns if 'proba' in col]].mean(axis=1)
predictions['voted_is_sale'] = np.where(predictions[[col for col in predictions.columns if 'predict' in col]].sum(axis=1)/len(models) <0.5, 0,1) 
predictions = predictions.join(df_y_final['is_sale'])
print (predictions)

In [ ]:
conn_df = pd.DataFrame(main_conn_df, columns = ['conn_id'])
print(main_conn_df)
print(conn_df)
predictions['conn_id'] = conn_df.loc[:,'conn_id']
print (predictions.head())

In [ ]:
cols_dict = {}
cols_dict['ordered_cols'] = ordered_cols
cols_dict['cat_cols'] = cat_cols
cols_dict['num_cols'] = num_cols

with open(model_dir+"/columns.txt", 'w') as f: 
  print(cols_dict, file = f)

with open(model_dir+"/columns.pkl", 'wb') as f:
  pickle.dump(cols_dict, f)
  print('dictionary saved successfully to file')

In [ ]:
groups_list = [10,20,40,80]
bins_dict = {}

for i in groups_list:

  predictions['PropensityGroup_bin'+str(i)], bins_dict[i] = pd.qcut(predictions["avg_proba"], retbins = True, q = i, labels=False)
  predictions.groupby(['PropensityGroup_bin' + str(i)])["avg_proba"].agg(["min","max","count"]).reset_index()#.to_csv(f" thresholds_{today_date}_all.csv", index = False)
  #print(predictions.groupby(["PropensityGroup_bin" + str(i)]) ["avg_proba"].agg(["min","max","count"]))

  bins_dict[i][0] = 0
  bins_dict[i][-1] = 1 
  print(bins_dict[i])

In [ ]:
with open(model_dir+"/bins.txt", 'w') as f: 
  print(bins_dict, file = f)

with open (model_dir+"/bins.pkl", 'wb') as f:
  pickle.dump(bins_dict, f)
  print('dictionary saved successfully to file')

with open(model_dir+"/explanation.txt", 'w') as f: 
  print(script_pars['explanation'], file = f)

In [ ]:
fig, axs = plt.subplots(figsize = (8,12), nrows = 4) 
fig.suptitle('Vertically stacked subplots') 
#axs[0].plot(x, y)
#axs[1].plot(x, y)

In [ ]:
for i in range(len(groups_list)):
  groups = predictions.groupby(["PropensityGroup_bin"+str(groups_list[i])])["is_sale"].agg(["count", "mean"])
  groups['mean'].plot(kind='bar', ax = axs[i])
  #ax_str.set_ylabel('Mean')
  #ax_str.set_xlabel('Propensity group')

  plt.grid()

In [ ]:
if ((script_pars['val_split'] == 'shuffle') and (script_pars['shuffle_test_size'] > 0)) or ((script_pars['val_split'] == 'lastN') and (script_pars['lastn_days']>0)): 
  
  predictions_val = pd.DataFrame()
  for i in range(len(models)):
    predictions_val['proba' + str(i)] = models[i].predict_proba (val_x)[:,1] 
    predictions_val['predict'+str(i)] = models[i].predict(val_x)

  predictions_val['avg_proba'] = predictions_val[[col for col in predictions_val.columns if 'proba' in col]].mean(axis=1)
  predictions_val['voted_is_sale'] = np.where(predictions_val[[col for col in predictions_val.columns if 'predict' in col]].sum(axis=1)/len(models) <0.5, 0,1) 
  predictions_val = predictions_val.join(val_y['is_sale'])

  conn_df_val = pd.DataFrame(val_conn_df, columns = ['conn_id']) 
  predictions_val['conn_id'] = conn_df_val.loc[:,'conn_id']

  fig, axs = plt.subplots(figsize = (8,12), nrows = 4)
  fig.suptitle('Vertically stacked subplots')

  for ii in range(len(groups_list)):

    predictions_val["PropensityGroup_bin" +str(groups_list[ii])] = pd.cut(predictions_val["avg_proba"], bins=bins_dict[groups_list[ii]], labels=False, include_lowest=True) 
    predictions_val.groupby(["PropensityGroup_bin "+str(groups_list[ii])])["avg_proba"].agg(["min","max", "count"]).reset_index()#.to_csv(f" thresholds_{today_date}_all.csv", index = False) 
    #print(predictions_val.groupby(["PropensityGroup_bin" + str(groups_list[ii])])["avg_proba"].agg(["min","max","count"]))

  groups_val = predictions_val.groupby(["PropensityGroup_bin"+str(groups_list[ii])])["is_sale"].agg(["count","mean"])
  groups_val['mean'].plot(kind='bar', ax = axs[ii])
  plt.grid()

In [ ]:
### I have 2 different pipelines, i am trying best to make them parallel, going through same operations
### But sometimes It is possible that 2 dataframes going into Fit and predict might be different
### Difference may be dtypes, difference may be format of values, like 1-2-3 was categories
### I saved data as csv, then read it, then i figure out that python recognized them as floats
### Converted into a format like 1.0, 2.0, 3.0 . Even though they are converted to object dtype
### They will be treated as unseen categories on predict pipeline. And I will see the probabilities as very low. 
### Also another sanity check below is changing one feature value at a time, and getting probabilities for them to check consistency
### Most of the time, only one change differs proba, but not drastically. Then this csv is read and tested on inference pipeline too.
### Expecting almost same proba values with predict_proba function at that pipeline too. It is a sanity check

val_test = val_x.iloc[0:10]
val_test_f = val_test.append(val_test).reset_index(drop = True)
val_test_f.loc[10, 'feat_a'] = 'NA'
val_test_f.loc[11, 'feat_b'] = 'NA'
val_test_f.loc[12, 'feat_c'] = 'NA'
val_test_f.loc[13, 'feat_d'] = 'NA'
val_test_f.loc[14, 'feat_e'] = 'NA'
val_test_f.loc[15, 'feat_f'] = 30
val_test_f.loc[16, 'feat_g'] = 1000
val_test_f.loc[17, 'feat_h'] = 0
val_test_f.loc[18, 'feat_j'] = 70
val_test_f.loc[19, 'feat_m'] = 'NA'

print(val_test_f.head(20))

val_test_f.to_csv(model_dir+"val_test.csv", index = False)

In [ ]:
predictions_val3 = pd.DataFrame()
for i in range(len(models)):
  predictions_val3['proba' + str(i)] = models[i].predict_proba (val_test_f)[:,1] 
  predictions_val3['predict'+str(i)] = models[i].predict(val_test_f)

predictions_val3['avg_proba'] = predictions_val3[[col for col in predictions_val3.columns if 'proba' in col]].mean(axis=1)
predictions_val3['voted_is_sale'] = np.where(predictions_val3[[col for col in predictions_val3.columns if 'predict' in col]].sum(axis=1)/len(models) <0.5, 0,1) 
predictions_val3 = predictions_val3.join(val_y['is_sale'])

conn_df_val = pd.DataFrame(val_conn_df, columns = ['conn_id']) 
predictions_val3['conn_id'] = conn_df_val.loc[:,'conn_id']

fig, axs = plt.subplots(figsize = (8,12), nrows = 4)
fig.suptitle('Vertically stacked subplots')

for ii in range(len(groups_list)):

  predictions_val3["PropensityGroup_bin" +str(groups_list[ii])] = pd.cut(predictions_val3["avg_proba"], bins=bins_dict[groups_list[ii]], labels=False, include_lowest=True) 
  predictions_val3.groupby(["PropensityGroup_bin "+str(groups_list[ii])])["avg_proba"].agg(["min","max", "count"]).reset_index()#.to_csv(f" thresholds_{today_date}_all.csv", index = False) 
  #print(predictions_val.groupby(["PropensityGroup_bin" + str(groups_list[ii])])["avg_proba"].agg(["min","max","count"]))

print(predictions_val3["PropensityGroup_bin40"].value_counts())
print(predictions_val3.head(20))

In [ ]:
if (len(outsamp_df) > 0) and (len(outsamp_y) > 0):

  predictions_outsamp = pd.DataFrame()
  for i in range(len(models)):
    predictions_outsamp['proba' + str(i)] = models[i].predict_proba(outsamp_df)[:,1]
    predictions_outsamp['predict' + str(i)] = models[i].predict(outsamp_df)

  predictions_outsamp['avg_proba'] = predictions_outsamp[[col for col in predictions_outsamp.columns if 'proba' in col]].mean(axis=1)
  predictions_outsamp['voted_is_sale'] = np.where(predictions_outsamp[[col for col in predictions_outsamp.columns if 'predict' in col]].sum(axis=1)/len(models) <0.5, 0,1) 
  predictions_outsamp = predictions_outsamp.join(outsamp_y['is_sale'])

  conn_df_outsamp = pd.DataFrame(outsamp_conn_df, columns = ['conn_id']) 
  predictions_outsamp['conn_id'] = conn_df_outsamp.loc[:,'conn_id']

  #fig, axs = plt.subplots(figsize = (8,12), nrows = 4)
  #fig.suptitle('Vertically stacked subplots')

  for ii in range(len(groups_list)):
    
    predictions_outsamp["PropensityGroup_bin" + str(groups_list[ii])] = pd.cut(predictions_outsamp["avg_proba"], bins=bins_dict[groups_list[ii]], labels=False, include_lowest=True) 
    predictions_outsamp.groupby(["PropensityGroup_bin"+str(groups_list[ii])])["avg_proba"].agg(["min", "max","count"]).reset_index()#.to_csv(f" thresholds_{today_date}_all.csv", index = False) 
    
    groups_outsamp = predictions_outsamp.groupby(["PropensityGroup_bin" +str(groups_list[ii])])["is_sale"].agg(["count", "mean"]) 
    #groups_outsamp['mean'].plot(kind='bar', ax = axs[ii])
    #plt.grid()

In [ ]:
print(predictions)
publish_df = predictions.loc[:, ['conn_id', 'PropensityGroup_bin10', 'PropensityGroup_bin20', 'PropensityGroup_bin40', 'PropensityGroup_bin80']]
publish_df_val = pd.concat([publish_df, predictions_val.loc[:, ['conn_id', 'PropensityGroup_binle', 'PropensityGroup_bin20', 'PropensityGroup_bin40', 'PropensityGroup_bin80']]], ignore_index=True)
publish_df_all = pd.concat([publish_df_val, predictions_outsamp.loc[:, ['conn_id', 'PropensityGroup_bin10', 'PropensityGroup_bin20', 'PropensityGroup_bin40', 'PropensityGroup_bin80']]], ignore_index=True) 
print(publish_df_all)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, fl_score, confusion_matrix 
for i in range(len(models)):
  #print("Model_"+str(i)+"_train AUC is (0:.3f}".format(roc_auc_score(predictions['is_sale'], predictions['predict'+str(i)]))) 
  #print("Model_"+str(i)+"_train Accuracy is (0:.3f}".format(accuracy_score(predictions['is_sale'], predictions['predict' + str(i)]))) 
  #print("Model "+str(i)+"_train Precision is (0:.3f}".format(precision_score(predictions['is_sale'], predictions['predict'+str(i)]))) 
  #print("Model_"+str(i)+"_train Recall is (0:.3f}".format(recall_score(predictions['is_sale'], predictions['predict' + str(i)]))) 
  #print("Model_"+str(i)+"_train F1 Score is (0:.3f}".format(fl_score(predictions['is_sale'], predictions['predict'+str(i)]))) 
  
  print("Model_"+str(i)+"_valid AUC is {0:.3f}".format(roc_auc_score(predictions_val['is_sale'], predictions_val['predict' + str(i)]))) 
  print("Model "+str(i)+"_valid Accuracy is {0:.3f}".format(accuracy_score(predictions_val['is_sale'], predictions_val['predict'+str(i)])))
  print("Model "+str(i)+"_valid Precision is (0:.3f}".format(precision_score(predictions_val['is_sale'], predictions_val['predict'+str(i)]))) 
  print("Model "+str(i)+"_valid Recall is (0:.3f}".format(recall_score(predictions_val['is_sale'], predictions_val['predict'+str(i)]))) 
  print("Model_"+str(i)+"_valid F1 Score is {0:.3f}".format(f1_score(predictions_val['is_sale'], predictions_val['predict'+str(i)]))) 
  
#plot_cm(y_true,y_pred, 'Confusion Matrix for LGB Model')

In [ ]:
def plot_cm(y_true, y_pred, title):
  figsize = (5, 5)
  y_pred = y_pred.astype (int)
  cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
  cm_sum = np.sum(cm, axis=1, keepdims=True)
  cm_perc = cm / cm_sum.astype(float) * 100
  annot= np.empty_like(cm).astype(str)
  nrows, ncols = cm.shape
  for i in range(nrows):
    for j in range(ncols):
      c = cm[i, j]
      p = cm_perc[i, j]
      if i == j:
        s = cm_sum[i]
        annot[i, j] = '%.1f%%\n\%d/%d' % (p, c, s)
      elif c == 0:
        annot[i, j] = ''
      else:
        annot[i, j] = '%.1f%%\n%d' % (p, c)
  cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true)) 
  cm.index.name = 'Actual'
  cm.columns.name = 'Predicted'
  fig, ax = plt.subplots(figsize=figsize)
  plt.title(title)
  sns.heatmap(cm, cmap="YLGnBu", annot=annot, fmt='', ax=ax)

In [ ]:
plot_cm(predictions['is_sale'], predictions['voted_is_sale'], 'Train_Confusion_Matrix') 
plot_cm(predictions_val['is_sale'], predictions_val['voted_is_sale'], 'Validation_Confusion_Matrix')

In [ ]:
publish_df_all['calltime'] = '2023-01-01 01:00:00'
publish_df_all['calltime'] = publish_df_all['calltime'].astype('datetime64')
print(publish_df_all.head())

In [ ]:
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.engine import url as sa_url

def create_mysql_engine(host, database, username, password, port='3307'):

  url = sa_url.URL(drivername= 'mysql+pymysql', host=host, port=port, database=database, username=username, password=password)
  print(url)
  return sa.create_engine(url)

engine=create_mysql_engine(ai['host'], 'smexplorerdata_fbb', ai['user'], ai['password'], ai['port'])

publish_df_all.to_sql(if_exists = 'replace', name= 'fbb_retention_propensity_groups_nobtn_nopivot', con = engine, schema= 'smexplorerdata_fbb', index = False)

In [ ]:
engine=create_mysql_engine(ai['host'], 'smexplorerdata_fbb', ai['user'], ai['password'], ai['port'])

with engine.connect() as con:
  con.execute('CREATE INDEX fbb_retention_propensity_groups_nobtn_nopivot_conn_id_IDX USING BTREE ON smexplorerdata_fbb.fbb_retention_propensity_groups_nobtn_nopivot (conn_id (16));')